# Load Vision Model

First we load in the data and our model into memory

**Important:** For this notebook to function it needs to be execuetd with `export LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1:/$LD_PRELOAD` else you will get a error. 
I included this in jupyter by adding a docker environment variable: `-e LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1`.

In [1]:
import numpy as np
import torch

In [2]:
from tinyyolov2 import TinyYoloV2
from utils.yolo import nms, filter_boxes
from utils.viz import display_result

# make an instance with 20 classes as output
net = TinyYoloV2(num_classes=20)

# load pretrained weights
sd = torch.load("voc_pretrained.pt")
net.load_state_dict(sd)

#put network in evaluation mode
net.eval()

TinyYoloV2(
  (pad): ReflectionPad2d((0, 1, 0, 1))
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=

# Define Camera Callback

In [3]:
from utils.camera import CameraDisplay
import time
import cv2
now = time.time()

In [4]:
# To control log spamming
frame_counter = 0  # Counts frames
debug_interval = 10  # Log every 30 frames

# How sensitive the model is 
confidence_val = 0.0001 # default: 0.1
overlap_val = 0.05 # default: 0.25

In [5]:
def get_predictions(image):
    global frame_counter
    img_shape = 320

    # Ensure the input tensor is correct
    if image.shape != (1, 3, 320, 320):
        raise ValueError(f"Invalid input shape: {image.shape}, expected (1, 3, 320, 320)")

    # Pass through the network
    with torch.no_grad():
        output = net(image)

    # Process YOLO output
    output = filter_boxes(output, confidence_val)

    output = nms(output, overlap_val)

    # Convert the tensor back to a NumPy array for OpenCV
    image_np = image.squeeze(0).permute(1, 2, 0).numpy()
    image_np = (image_np * 255).astype(np.uint8)  # Scale back to 0-255 and convert to uint

    if output:
        if frame_counter % debug_interval == 0:
            print(f"Frame {frame_counter}: output not empty.")
        bboxes = torch.stack(output, dim=0)

        if frame_counter % debug_interval == 0:
            print(f"Frame {frame_counter}: Number of bboxes: {bboxes.shape[1]}")
        
        for i in range(bboxes.shape[1]):
            # Extract box information
            cx = bboxes[0, i, 0].item()
            cy = bboxes[0, i, 1].item()
            w = bboxes[0, i, 2].item()
            h = bboxes[0, i, 3].item()
            conf = bboxes[0, i, 4].item()
            class_id = int(bboxes[0, i, 5].item())
            class_name = num_to_class(class_id)

            if frame_counter % debug_interval == 0:
                print(f"Detection {i}: Center=({cx:.2f}, {cy:.2f}), Size=({w:.2f}, {h:.2f}), "
                      f"conf={conf:.2f}, Class={class_name} ({class_id})")

            # Convert normalized coordinates to pixel coordinates
            x_min = int((cx - w / 2) * img_shape)
            y_min = int((cy - h / 2) * img_shape)
            x_max = int((cx + w / 2) * img_shape)
            y_max = int((cy + h / 2) * img_shape)

            # Verify if coordinates are within bounds
            if x_min < 0 or y_min < 0 or x_max > img_shape or y_max > img_shape:
                print(f"Detection {i}: Bounding box out of bounds! ({x_min}, {y_min}, {x_max}, {y_max})")

            # Draw rectangle on the image
            cv2.rectangle(
                image_np,
                (x_min, y_min),
                (x_max, y_max),
                (0, 0, 255),  # Red
                2
            )

            # Add label text
            label = f"{class_name} {conf:.2f}"
            cv2.putText(
                image_np,
                label,
                (x_min, y_min - 10),  # Slightly above the rectangle
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,  # Font scale
                (0, 0, 255),  # Red
                1  # Thickness
            )
    else:
        if frame_counter % debug_interval == 0:
            print(f"Frame {frame_counter}: No detections.")

    return image_np

In [6]:
def callback(image):
    global now, frame_counter

    frame_counter += 1
    if image is None:
        raise ValueError("Received empty frame from the camera.")

    # Resize and preprocess the image
    img_resized = cv2.resize(image, (320, 320))
    img_tensor = torch.tensor(img_resized, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

    # Predict and visualize
    img_with_predictions = get_predictions(img_tensor)

    # Add FPS to the image
    fps = f"{int(1 / (time.time() - now))}"
    now = time.time()
    cv2.putText(img_with_predictions, f"fps={fps}", (2, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 0), 2, cv2.LINE_AA)

    return img_with_predictions

In [7]:
# Initialize the camera with the callback
cam = CameraDisplay(callback)

Initializing camera...


(Argus) Error Timeout:  (propagating from src/rpc/socket/client/SocketClientDispatch.cpp, function openSocketConnection(), line 219)
(Argus) Error Timeout: Cannot create camera provider (in src/rpc/socket/client/SocketClientDispatch.cpp, function createCameraProvider(), line 106)
Error generated. /dvs/git/dirty/git-master_linux/multimedia/nvgstreamer/gst-nvarguscamera/gstnvarguscamerasrc.cpp, execute:735 Failed to create CameraProvider
[ WARN:0@60.664] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


RuntimeError: Could not initialize camera.  Please see error trace.

# Camera Loop

In [ ]:
# The camera stream can be started with cam.start()
# The callback gets asynchronously called (can be stopped with cam.stop())
cam.start()

Execute below, to stop camera loop.

----------------------------------------------------------------

In [31]:
# The camera should always be stopped and released for a new camera is instantiated (calling CameraDisplay(callback) again)
cam.stop()
cam.release()

Camera released
